In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-128142")
exp = Experiment(workspace=ws, name="quick-starts-ws-128142")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

ModuleNotFoundError: No module named 'azureml'

In [2]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
from azureml.core.compute_target import ComputeTargetException

   # Choose a name for your CPU cluster
cpu_cluster_name = "cpu-cluster"

   # Verify that cluster does not exist already
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',
                                                              max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Found existing cluster, use it.

Running


In [8]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform,loguniform,choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    "--C": choice(0.5,1,1.5),
    "--max_iter":choice(100,150,125)
})

# Specify a Policy
policy = BanditPolicy(slack_factor=0.1,evaluation_interval=1,delay_evaluation=5)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(
            source_directory = ".",
            compute_target = cpu_cluster_name,
            entry_script='train.py',
        )

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(
                        estimator=est,
                        hyperparameter_sampling=ps,
                        policy=policy,
                        primary_metric_name="Accuracy",
                        primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                        max_total_runs=10,
                        max_concurrent_runs=4
                    )

In [7]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
experiment = Experiment(workspace=ws,name="hyperdrive_experiment")
hyper_run = experiment.submit(hyperdrive_config)
### YOUR CODE HERE ###

WARNING - If 'script' has been provided here and a script file name has been specified in 'run_config', 'script' provided in ScriptRunConfig initialization will take precedence.


In [ ]:
RunDetails(hyper_run).show()

In [14]:
import joblib
# Get your best run and save the model from that run.
best_run = hyper_run.get_best_run_by_primary_metric().get_metrics()
joblib.dump(best_run,'best_run_hyperdrive')
### YOUR CODE HERE ###

['best_run_hyperdrive']

In [3]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

automlds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")
ds = automlds.to_pandas_dataframe()
print(ds)
### YOUR CODE HERE ###

       age          job  marital          education  default housing loan  \
0       57   technician  married        high.school       no      no  yes   
1       55      unknown  married            unknown  unknown     yes   no   
2       33  blue-collar  married           basic.9y       no      no   no   
3       36       admin.  married        high.school       no      no   no   
4       27    housemaid  married        high.school       no     yes   no   
...    ...          ...      ...                ...      ...     ...  ...   
32945   56    housemaid  married           basic.4y       no      no  yes   
32946   37   management  married  university.degree       no      no  yes   
32947   26       admin.   single  university.degree       no      no   no   
32948   31  blue-collar   single           basic.9y       no      no   no   
32949   39    housemaid  married           basic.4y       no      no   no   

         contact month day_of_week  ...  campaign  pdays  previous  \
0    

In [4]:
from train import clean_data
from sklearn.model_selection import train_test_split
import pandas as pd

# Use the clean_data function to clean your data.

In [5]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=20,
    task='classification',
    enable_onnx_compatible_models=True,
    primary_metric='accuracy',
    training_data=ds,
    label_column_name="y",
    n_cross_validations=4)

In [6]:
# Submit your automl run
experiment = Experiment(workspace=ws,name="automl-run")
automl_run = experiment.submit(automl_config)
### YOUR CODE HERE ###

WARNING - Running on local machine. Note that local runs always run synchronously even if you use the parameter 'show_output=False'


In [10]:
# Retrieve and save your best automl model.
best_run, fitted_model = automl_run.get_output(return_onnx_model=True)
RunDetails(automl_run).show()
### YOUR CODE HERE ###

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [12]:
from azureml.automl.runtime.onnx_convert import OnnxConverter
file_path = "./best_model.onnx"
OnnxConverter.save_onnx_model(fitted_model, file_path)

In [ ]:
cpu_cluster.delete()